# 1. Intro to XML

> How to (and why) convert XML to JSON

In [1]:
#| hide
%load_ext autoreload
%autoreload 2

In [2]:
#| hide
from fastcore.test import *
from nbdev.showdoc import *

## XML
XML is a markup language that is used to store and transport data. It is both human-readable and machine-readable.  It is flexible, allowing to define new tags and attributes as needed.  It is portable and platform-independent, and can be read as plain text or in a variety of editors and parsers.

However, it's verbose, complex, hard to debug, and not queryable.  The goal of this project is to convert XMLs to (semi)relational data formats that can support diverse data warehousing and reporting solutions.

In one approach, we would pursue converting the non-tabular XML data into something tabular such as CSV.  However, with 100+ fields and attributes, it is laborious to spell out all the possibly useful data elements.  We don't want to discard any information yet either.  Luckily, modern data ecosystems have been exposed to this problem for long enough, and there is a variety of solutions to query branched, semi-structured data.  Many SQL-like query engines and reporting tools (**including Tableau**) can work with JSON format, which will be the target of the initial XML processing work.

## XML Example: Mass-Spectrometry

Let's have a look at a sample XML file.  In this case, XML format is used to store mass-spectrometry data (`.mzML` files).  It has no relevance to TEG, except for the similarity in how the information is organized.  Just like in TEG files, there is the spectral data, and the data about the data (metadata).  We need both, but we need treat them differently.

In [3]:
import json
import requests
import tempfile
import xmltodict

Let's retrieve sample XML data from the internet using an HTTP request.  The result, `xml_data`, is a bytestring with the contents of the downloaded XML file:

In [4]:
xml_url = "https://raw.githubusercontent.com/ProteoWizard/pwiz/master/example_data/tiny.pwiz.1.1.1.mzML"
xml_data = requests.get(xml_url).content
xml_data[:2048]

b'<?xml version="1.0" encoding="ISO-8859-1"?>\n<indexedmzML xmlns="http://psi.hupo.org/ms/mzml" xmlns:xsi="http://www.w3.org/2001/XMLSchema-instance" xsi:schemaLocation="http://psi.hupo.org/ms/mzml http://psidev.info/files/ms/mzML/xsd/mzML1.1.1_idx.xsd">\n  <mzML xmlns="http://psi.hupo.org/ms/mzml" xmlns:xsi="http://www.w3.org/2001/XMLSchema-instance" xsi:schemaLocation="http://psi.hupo.org/ms/mzml http://psidev.info/files/ms/mzML/xsd/mzML1.1.0.xsd" id="urn:lsid:psidev.info:mzML.instanceDocuments.tiny.pwiz" version="1.1.0">\n    <cvList count="2">\n      <cv id="MS" fullName="Proteomics Standards Initiative Mass Spectrometry Ontology" version="2.33.1" URI="http://psidev.cvs.sourceforge.net/*checkout*/psidev/psi/psi-ms/mzML/controlledVocabulary/psi-ms.obo"/>\n      <cv id="UO" fullName="Unit Ontology" version="11:02:2010" URI="http://obo.cvs.sourceforge.net/*checkout*/obo/obo/ontology/phenotype/unit.obo"/>\n    </cvList>\n    <fileDescription>\n      <fileContent>\n        <cvParam cvRe

In [5]:
#| hide
test_eq(xml_data[:64], b'<?xml version="1.0" encoding="ISO-8859-1"?>\n<indexedmzML xmlns="')

### Reading as a dictionary and converting to JSON
Reading with `xmltodict` parser is fast and yields a python dictionary, which can be readily converted to JSON.  

There are several reasons why JSON format should work for us to stage this data.  Among others, it's easier to see the structure in a pretty-printed JSON:

In [6]:
xml_dict = xmltodict.parse(xml_data, attr_prefix="")
print(json.dumps(xml_dict, indent=4)[:2048])

{
    "indexedmzML": {
        "xmlns": "http://psi.hupo.org/ms/mzml",
        "xmlns:xsi": "http://www.w3.org/2001/XMLSchema-instance",
        "xsi:schemaLocation": "http://psi.hupo.org/ms/mzml http://psidev.info/files/ms/mzML/xsd/mzML1.1.1_idx.xsd",
        "mzML": {
            "xmlns": "http://psi.hupo.org/ms/mzml",
            "xmlns:xsi": "http://www.w3.org/2001/XMLSchema-instance",
            "xsi:schemaLocation": "http://psi.hupo.org/ms/mzml http://psidev.info/files/ms/mzML/xsd/mzML1.1.0.xsd",
            "id": "urn:lsid:psidev.info:mzML.instanceDocuments.tiny.pwiz",
            "version": "1.1.0",
            "cvList": {
                "count": "2",
                "cv": [
                    {
                        "id": "MS",
                        "fullName": "Proteomics Standards Initiative Mass Spectrometry Ontology",
                        "version": "2.33.1",
                        "URI": "http://psidev.cvs.sourceforge.net/*checkout*/psidev/psi/psi-ms/mzML/contr

JSON data can be saved as a file:

In [7]:
with tempfile.NamedTemporaryFile("w", delete=False) as tf:
    tf.write(json.dumps(xml_dict, indent=4))

And loaded from a file:

In [8]:
with open(tf.name) as f:
    print(f.read(128))

{
    "indexedmzML": {
        "xmlns": "http://psi.hupo.org/ms/mzml",
        "xmlns:xsi": "http://www.w3.org/2001/XMLSchema-in


## Querying Non-Tabular Data: DuckDB

DuckDB is a powerful, performant in-memory, in-process analytical database.  We can use it to mimick a data warehousing environment, and write queries against JSON files directly.  Tableau also can use JSON files as data sources.

In [9]:
import duckdb

DuckDB allows to query JSON files directly:

In [10]:
con = duckdb.connect()
q = f"""
SELECT * FROM read_json_auto("{tf.name}")
"""
con.execute(q).df()

,indexedmzML
0,"{'xmlns': 'http://psi.hupo.org/ms/mzml', 'xmln..."


Let's retrieve a few specific element: a string, a nested dictionary, and a list:

In [11]:
q = f"""
SELECT
  indexedmzML.mzML.run.startTimeStamp AS run_timestamp
  ,indexedmzML.mzML.scanSettingsList AS scan_settings
  ,indexedmzML.mzML.run.chromatogramList.chromatogram[1].binaryDataArrayList.binaryDataArray AS data_array
FROM read_json_auto('{tf.name}')
"""
df = con.execute(q).df()
df

,run_timestamp,scan_settings,data_array
0,2007-06-27T15:23:45.00035,"{'count': 1, 'scanSettings': {'id': 'tiny_x002...","[{'encodedLength': 160, 'dataProcessingRef': '..."


In [12]:
df.scan_settings

0    {'count': 1, 'scanSettings': {'id': 'tiny_x002...
Name: scan_settings, dtype: object

In [13]:
[data["binary"] for data in df.data_array[0]]

['AAAAAAAAAAAAAAAAAADwPwAAAAAAAABAAAAAAAAACEAAAAAAAAAQQAAAAAAAABRAAAAAAAAAGEAAAAAAAAAcQAAAAAAAACBAAAAAAAAAIkAAAAAAAAAkQAAAAAAAACZAAAAAAAAAKEAAAAAAAAAqQAAAAAAAACxA',
 'AAAAAAAALkAAAAAAAAAsQAAAAAAAACpAAAAAAAAAKEAAAAAAAAAmQAAAAAAAACRAAAAAAAAAIkAAAAAAAAAgQAAAAAAAABxAAAAAAAAAGEAAAAAAAAAUQAAAAAAAABBAAAAAAAAACEAAAAAAAAAAQAAAAAAAAPA/']

Alternatively, the lists can be unnested right in the query.  In this case, the first two data elements are exactly the same, illustrating **one-to-many** relationship between run metadata and the actual data.

In [14]:
q = f"""
SELECT
  indexedmzML.mzML.run.startTimeStamp AS run_timestamp
  ,indexedmzML.mzML.scanSettingsList AS scan_settings
  ,UNNEST(indexedmzML.mzML.run.chromatogramList.chromatogram[1].binaryDataArrayList.binaryDataArray).binary AS data
FROM read_json_auto('{tf.name}')
"""
df = con.execute(q).df()
df

,run_timestamp,scan_settings,data
0,2007-06-27T15:23:45.00035,"{'count': 1, 'scanSettings': {'id': 'tiny_x002...",AAAAAAAAAAAAAAAAAADwPwAAAAAAAABAAAAAAAAACEAAAA...
1,2007-06-27T15:23:45.00035,"{'count': 1, 'scanSettings': {'id': 'tiny_x002...",AAAAAAAALkAAAAAAAAAsQAAAAAAAACpAAAAAAAAAKEAAAA...


## Separating dicts and lists
JSON format is a collection of `key:value` pairs, where the `value` can be one of:  
- a single value (starts with "`"`" or a number, ends with comma);  
- a list (starting with "`[`");  
- another branch (starts with "`{`").  

In most scenarios, dicts/single values will describe the run, and lists will be associated with repeated arrays of measurements.  It makes sense to pull them apart - for both data management and performance reasons (we eventually intend to read hundreds of thousands of files!)

In [15]:
# to be continued...

In [16]:
#| hide
import nbdev; nbdev.nbdev_export()